In [ ]:
import os
import sys

In [4]:
import pandas as pd
import glob
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline 


# Seaborn visualization library
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression


from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier

import scipy

In [17]:
pd.set_option('display.max_columns', 1000)
pd.options.display.max_rows=1000

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

plt.rcParams.update({'figure.max_open_warning': 0})


In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from scripts.supervised_prep import test_print

In [3]:
test_print(1)

1


In [9]:
inroot = r"../data/input/07 Samsung UX Index - Web App Implementation/"
fname_data = inroot + r"Samsung UX Index Survey_Data.csv"
df_data = pd.read_csv(fname_data)
fname_vaxmap = inroot + r"Samsung UX Index Survey_Datamap.xlsx"
df_varmap = pd.read_excel(fname_vaxmap, header=1, sheet_name=0)
df_valmap = pd.read_excel(fname_vaxmap, header=1, sheet_name=1)



/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1218) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
SuperLabels = ['drivers', 'brandlovetrust', 'loyaltymetrics', 'overallquality',
          'usagemetrics', 'activitiesximportance','activitiexsquality',
          'activitiesxsatisfaction','activitiesxdrivers',
           'upgradetrans','ECGexpect']

SuperLabels = [x+"xallATTR" for x in SuperLabels]

SuperList = [['qxdrivers_'+str(i+1) for i in range(34)],
             [ x for x in df_data.columns if 'qxbrandx' in x ],
            ['qxadvocacy01_1',    'qxadvocacy02_1',    'qxretention_1',    'qxenrichment_1'],
            ['qxoverallxqualityxindicators_'+str(i+1) for i in range(4)],
            [ x for x in df_data.columns if 'qxactivitiesximportance' in x ],
            [ x for x in df_data.columns if 'qxactivitiesxqualityxindicators' in x ],
            [ x for x in df_data.columns if 'qxactivitiesxsatisfaction' in x ],
            [ x for x in df_data.columns if 'qxactivitiesxdrivers' in x ] ,
         ['qxupgrade01_1', 'qxupgrade01_2', 'qxupgrade01_3', 'qxupgrade01_4', 
          'qxupgrade01_5', 'qxupgrade01_6', 'qxupgrade01_7', 'qxupgrade01_8', 
          'qxtransition_1'],
[ 'att01_1','att01_2','att01_3','att02_1','att02_2','soc03','soc04_1',
          'soc04_2','soc04_3','ret06_1','qxexpectations' ]
            ]

varlist_Z  = ['d1',    'd3_1',    'd3_2',    'd3_3',    'd3_4',    'd4', 'd6',    'd7_1',    
            'd7_2',    'd7_3',    'd7_4',    'd7_5',    'd7_97',    'd7_99','qxcurrentxos',
              'qxactivitiesxrecency_1', 'qxactivitiesxrecency_2', 'qxactivitiesxrecency_3', 
              'qxactivitiesxrecency_4', 'qxactivitiesxrecency_5', 'qxactivitiesxrecency_6', 
              'qxactivitiesxrecency_7', 'qxactivitiesxrecency_8', 'qxactivitiesxrecency_9', 
              'qxactivitiesxrecency_10', 'qxactivitiesxrecency_11', 'qxactivitiesxrecency_12', 
              'qxactivitiesxrecency_13', 'qxactivitiesxrecency_14', 'qxactivitiesxrecency_15', 
              'qxactivitiesxrecency_16', 'qxactivitiesxrecency_17', 'qxactivitiesxrecency_18', 
              'qxactivitiesxrecency_19', 'qxactivitiesxrecency_20', 'qxactivitiesxrecency_21', 
              'qxactivitiesxrecency_22', 'qxactivitiesxrecency_23', 'qxactivitiesxrecency_24'
              ]

In [13]:
any('activitiesxrecency' in sublist for sublist in varlist_Z) 

True

In [21]:
df_data.filter
[df_data.columns.contains('activitiesxrecency')]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  """Entry point for launching an IPython kernel.


KeyError: False

In [18]:
df_varmap[df_varmap.Variable.str.contains('activitiesxrecency')][['Variable','Label']]

,Variable,Label
37,qxactivitiesxrecency_1,"Q_Activities_Recency. Using speaker phone (including all aspects of it, such as dialing, quality of sound, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps"
38,qxactivitiesxrecency_2,"Q_Activities_Recency. Charging battery (including wired charge, wireless charge, different charge options, the accuracy of the battery indicator, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience"
39,qxactivitiesxrecency_3,"Q_Activities_Recency. Transferring files from smartphone to a computer - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps only, and don’t consider your experienc"
40,qxactivitiesxrecency_4,"Q_Activities_Recency. Unlocking the smartphone (e.g., via password, fingerprint, facial or voice recognition) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps on"
41,qxactivitiesxrecency_5,"Q_Activities_Recency. Viewing screen (e.g., watching video, viewing pictures, playing games) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps only, and don’t c"
42,qxactivitiesxrecency_6,"Q_Activities_Recency. Setting up multiple windows / split screen - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps only, and don’t consider your experience with"
43,qxactivitiesxrecency_7,"Q_Activities_Recency. Organizing your home screen (moving apps, creating folders) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps only, and don’t consider you"
44,qxactivitiesxrecency_8,"Q_Activities_Recency. Using your quick settings (e.g., toggle WiFi, Airplane mode, brightness, volume, flashlight) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer ap"
45,qxactivitiesxrecency_9,"Q_Activities_Recency. Customizing settings (e.g., changing wallpaper, changing security/privacy requirements, etc.) - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer a"
46,qxactivitiesxrecency_10,"Q_Activities_Recency. Setting up and managing notifications - When was the last time you did each of the following on your [MODEL]? Please think of your experience with the preinstalled manufacturer apps only, and don’t consider your experience with alte"
